# Machine Learning Assignment - KNN, SVM and DT Experiments

This notebook implements experiments with KNN, SVM, and Decision Trees for both classification (Breast Cancer Wisconsin Diagnostic) and regression (Bike Sharing) problems.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_curve, auc
from sklearn import svm, tree
import time
import requests
from io import StringIO

# Set random seed for reproducibility
np.random.seed(42)

## Data Loading and Preprocessing

In [ ]:
# Load Breast Cancer Wisconsin Dataset
breast_cancer_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
response = requests.get(breast_cancer_url)
data = StringIO(response.text)

# Define column names
# ID, diagnosis (M = malignant, B = benign), and 30 feature columns
column_names = ['id', 'diagnosis'] + [f'feature_{i}' for i in range(1, 31)]
breast_cancer_df = pd.read_csv(data, header=None, names=column_names)

# Convert diagnosis to binary (1 for Malignant, 0 for Benign)
breast_cancer_df['diagnosis'] = breast_cancer_df['diagnosis'].map({'M': 1, 'B': 0})

# Drop ID column
breast_cancer_df = breast_cancer_df.drop('id', axis=1)

# Display first few rows of the dataset
print("Breast Cancer Dataset Preview:")
breast_cancer_df.head()

In [ ]:
# Load Bike Sharing Dataset
bike_sharing_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"

# For simplicity, let's download locally and read as pandas
# In Colab, you could use:
# !wget -O bike-sharing.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
# !unzip -o bike-sharing.zip
# bike_sharing_df = pd.read_csv('day.csv')

# For this notebook, we'll use a direct approach:
import zipfile
import io
import urllib.request

with urllib.request.urlopen(bike_sharing_url) as response:
    with zipfile.ZipFile(io.BytesIO(response.read())) as zip_ref:
        with zip_ref.open('day.csv') as f:
            bike_sharing_df = pd.read_csv(f)

# Display first few rows of the dataset
print("Bike Sharing Dataset Preview:")
bike_sharing_df.head()

In [ ]:
# Preprocess Breast Cancer Dataset
X_cancer = breast_cancer_df.drop('diagnosis', axis=1)
y_cancer = breast_cancer_df['diagnosis']

# Scale features
scaler_cancer = StandardScaler()
X_cancer_scaled = scaler_cancer.fit_transform(X_cancer)

# Preprocess Bike Sharing Dataset
# Remove instant, dteday and casual/registered (as cnt is their sum)
bike_sharing_df = bike_sharing_df.drop(['instant', 'dteday', 'casual', 'registered'], axis=1)

# Define features and target
X_bike = bike_sharing_df.drop('cnt', axis=1)
y_bike = bike_sharing_df['cnt']

# Scale features
scaler_bike = StandardScaler()
X_bike_scaled = scaler_bike.fit_transform(X_bike)

## Part 1: KNN Classifier with Euclidean Distance (K=3)

In this part, we implement a KNN classifier with Euclidean distance and test it on the Breast Cancer dataset.

In [ ]:
class KNNClassifier:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self
    
    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))
    
    def manhattan_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))
    
    def get_distance(self, x1, x2):
        if self.distance_metric == 'euclidean':
            return self.euclidean_distance(x1, x2)
        elif self.distance_metric == 'manhattan':
            return self.manhattan_distance(x1, x2)
        else:
            raise ValueError(f"Unknown distance metric: {self.distance_metric}")
    
    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)
    
    def _predict(self, x):
        # Calculate distances between x and all examples in the training set
        distances = [self.get_distance(x, x_train) for x_train in self.X_train]
        
        # Get k nearest samples, labels
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        # Return most common class
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common

In [ ]:
def evaluate_knn_classifier(X, y, k=3, distance_metric='euclidean', n_folds=6):
    # Initialize KFold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Initialize metrics storage
    accuracies = []
    conf_matrices = []
    times = []
    
    # For each fold
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train classifier
        start_time = time.time()
        clf = KNNClassifier(k=k, distance_metric=distance_metric)
        clf.fit(X_train, y_train)
        
        # Predict
        y_pred = clf.predict(X_test)
        end_time = time.time()
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        execution_time = end_time - start_time
        
        # Store metrics
        accuracies.append(accuracy)
        conf_matrices.append(conf_matrix)
        times.append(execution_time)
        
        # Print metrics for the first fold (as required)
        if fold == 0:
            print(f"Fold 1 results:")
            print(f"Training/test split: {len(X_train)}/{len(X_test)} samples")
            print(f"Accuracy: {accuracy:.4f}")
            print(f"Confusion Matrix:")
            print(conf_matrix)
            print(f"Execution time: {execution_time:.4f} seconds")
    
    # Print average metrics
    print("\nAverage results across all folds:")
    print(f"Average Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"Average Execution time: {np.mean(times):.4f} ± {np.std(times):.4f} seconds")
    
    # Return all metrics
    return {
        'accuracies': accuracies,
        'conf_matrices': conf_matrices,
        'times': times
    }

In [ ]:
# Run KNN Classifier with Euclidean distance
print("KNN Classifier with Euclidean Distance (K=3) on Breast Cancer Dataset:")
knn_classifier_results = evaluate_knn_classifier(
    X_cancer_scaled, 
    y_cancer.values, 
    k=3, 
    distance_metric='euclidean',
    n_folds=6
)

### Part 1 Results Summary

The custom-implemented KNN classifier with Euclidean distance (K=3) has been evaluated on the Breast Cancer Wisconsin dataset using 6-fold cross-validation. The results show the classifier's performance in terms of accuracy, confusion matrices, and execution time.

The confusion matrix interpretation:
- True Negatives (TN): Correctly predicted benign samples
- False Positives (FP): Benign samples incorrectly predicted as malignant
- False Negatives (FN): Malignant samples incorrectly predicted as benign
- True Positives (TP): Correctly predicted malignant samples

## Part 2: KNN Regressor with Manhattan Distance (K=3)

In this part, we implement a KNN regressor with Manhattan distance and test it on the Bike Sharing dataset.

In [ ]:
class KNNRegressor:
    def __init__(self, k=3, distance_metric='manhattan'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self
    
    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))
    
    def manhattan_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))
    
    def get_distance(self, x1, x2):
        if self.distance_metric == 'euclidean':
            return self.euclidean_distance(x1, x2)
        elif self.distance_metric == 'manhattan':
            return self.manhattan_distance(x1, x2)
        else:
            raise ValueError(f"Unknown distance metric: {self.distance_metric}")
    
    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)
    
    def _predict(self, x):
        # Calculate distances between x and all examples in the training set
        distances = [self.get_distance(x, x_train) for x_train in self.X_train]
        
        # Get k nearest samples, labels
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_targets = [self.y_train[i] for i in k_indices]
        
        # Return mean of k nearest neighbors
        return np.mean(k_nearest_targets)

In [ ]:
def evaluate_knn_regressor(X, y, k=3, distance_metric='manhattan', n_folds=6):
    # Initialize KFold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Initialize metrics storage
    mse_scores = []
    r2_scores = []
    times = []
    
    # For each fold
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train regressor
        start_time = time.time()
        reg = KNNRegressor(k=k, distance_metric=distance_metric)
        reg.fit(X_train, y_train)
        
        # Predict
        y_pred = reg.predict(X_test)
        end_time = time.time()
        
        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        execution_time = end_time - start_time
        
        # Store metrics
        mse_scores.append(mse)
        r2_scores.append(r2)
        times.append(execution_time)
        
        # Print metrics for the first fold (as required)
        if fold == 0:
            print(f"Fold 1 results:")
            print(f"Training/test split: {len(X_train)}/{len(X_test)} samples")
            print(f"Mean Squared Error (MSE): {mse:.4f}")
            print(f"R² Score: {r2:.4f}")
            print(f"Execution time: {execution_time:.4f} seconds")
            
            # Plot actual vs predicted for the first fold
            plt.figure(figsize=(10, 6))
            plt.scatter(y_test, y_pred, alpha=0.5)
            plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
            plt.xlabel('Actual')
            plt.ylabel('Predicted')
            plt.title('KNN Regressor: Actual vs Predicted (Fold 1)')
            plt.show()
    
    # Print average metrics
    print("\nAverage results across all folds:")
    print(f"Average MSE: {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")
    print(f"Average R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
    print(f"Average Execution time: {np.mean(times):.4f} ± {np.std(times):.4f} seconds")
    
    # Return all metrics
    return {
        'mse_scores': mse_scores,
        'r2_scores': r2_scores,
        'times': times
    }

In [ ]:
# Run KNN Regressor with Manhattan distance
print("KNN Regressor with Manhattan Distance (K=3) on Bike Sharing Dataset:")
knn_regressor_results = evaluate_knn_regressor(
    X_bike_scaled, 
    y_bike.values, 
    k=3, 
    distance_metric='manhattan',
    n_folds=6
)

### Part 2 Results Summary

The custom-implemented KNN regressor with Manhattan distance (K=3) has been evaluated on the Bike Sharing dataset using 6-fold cross-validation. The results show the regressor's performance in terms of Mean Squared Error (MSE), R² score, and execution time.

The scatter plot shows the relationship between actual and predicted values for the first fold, with the red dashed line representing perfect predictions. This helps visualize the model's performance.

## Part 3: Linear SVM Classifier

In this part, we implement a classifier based on linear SVM and test it on the Breast Cancer dataset.

In [ ]:
def evaluate_svm_classifier(X, y, n_folds=6):
    # Initialize KFold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Initialize metrics storage
    accuracies = []
    conf_matrices = []
    times = []
    roc_aucs = []
    tprs = []
    mean_fpr = np.linspace(0, 1, 100)
    optimal_thresholds = []
    
    # For plotting ROC curve
    plt.figure(figsize=(10, 8))
    
    # For each fold
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train classifier
        start_time = time.time()
        clf = svm.SVC(kernel='linear', probability=True, random_state=42)
        clf.fit(X_train, y_train)
        
        # Get decision scores (distance from hyperplane)
        y_scores = clf.decision_function(X_test)
        
        # Get probabilities for ROC curve
        y_probs = clf.predict_proba(X_test)[:, 1]
        
        # Predict with default threshold
        y_pred = clf.predict(X_test)
        end_time = time.time()
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        execution_time = end_time - start_time
        
        # Calculate ROC curve and area
        fpr, tpr, thresholds = roc_curve(y_test, y_probs)
        roc_auc = auc(fpr, tpr)
        roc_aucs.append(roc_auc)
        
        # Interpolate TPR values for mean curve calculation
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        
        # Find the optimal threshold using Youden's J statistic (max(TPR - FPR))
        j_scores = tpr - fpr
        optimal_idx = np.argmax(j_scores)
        optimal_threshold = thresholds[optimal_idx]
        optimal_thresholds.append(optimal_threshold)
        
        # Apply optimal threshold to get predictions
        y_pred_optimal = (y_probs >= optimal_threshold).astype(int)
        optimal_acc = accuracy_score(y_test, y_pred_optimal)
        optimal_conf_matrix = confusion_matrix(y_test, y_pred_optimal)
        
        # Plot ROC curve for each fold
        plt.plot(fpr, tpr, lw=1, alpha=0.3,
                 label=f'ROC fold {fold+1} (AUC = {roc_auc:.2f})')
        
        # Store metrics
        accuracies.append(optimal_acc)  # Using accuracy with optimal threshold
        conf_matrices.append(optimal_conf_matrix)  # Using confusion matrix with optimal threshold
        times.append(execution_time)
        
        # Print metrics for the first fold (as required)
        if fold == 0:
            print(f"Fold 1 results:")
            print(f"Training/test split: {len(X_train)}/{len(X_test)} samples")
            print(f"Default Accuracy: {accuracy:.4f}")
            print(f"Default Confusion Matrix:")
            print(conf_matrix)
            print(f"\nOptimal Threshold: {optimal_threshold:.4f}")
            print(f"Optimal Accuracy: {optimal_acc:.4f}")
            print(f"Optimal Confusion Matrix:")
            print(optimal_conf_matrix)
            print(f"ROC AUC: {roc_auc:.4f}")
            print(f"Execution time: {execution_time:.4f} seconds")
    
    # Plot mean ROC curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(roc_aucs)
    plt.plot(mean_fpr, mean_tpr, 'b-',
             label=f'Mean ROC (AUC = {mean_auc:.2f} ± {std_auc:.2f})',
             lw=2, alpha=0.8)

    # Plot standard deviation
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.2,
                      label=f'± 1 std. dev.')
    
    # Plot diagonal
    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
    
    # Set plot properties
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for Linear SVM Classifier')
    plt.legend(loc='lower right')
    plt.show()
    
    # Print average metrics
    print("\nAverage results across all folds:")
    print(f"Average Accuracy (with optimal thresholds): {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"Average ROC AUC: {np.mean(roc_aucs):.4f} ± {np.std(roc_aucs):.4f}")
    print(f"Average Optimal Threshold: {np.mean(optimal_thresholds):.4f} ± {np.std(optimal_thresholds):.4f}")
    print(f"Average Execution time: {np.mean(times):.4f} ± {np.std(times):.4f} seconds")
    
    # Return all metrics
    return {
        'accuracies': accuracies,
        'conf_matrices': conf_matrices,
        'roc_aucs': roc_aucs,
        'optimal_thresholds': optimal_thresholds,
        'times': times
    }

In [ ]:
# Run Linear SVM Classifier
print("Linear SVM Classifier on Breast Cancer Dataset:")
svm_classifier_results = evaluate_svm_classifier(
    X_cancer_scaled, 
    y_cancer.values,
    n_folds=6
)

### Part 3 Results Summary

The linear SVM classifier has been evaluated on the Breast Cancer Wisconsin dataset using 6-fold cross-validation. The results show the classifier's performance in terms of accuracy, confusion matrices, ROC curves, and execution time.

As required, we found the best threshold for the SVM output using Youden's J statistic, which maximizes the difference between the true positive rate and false positive rate. This optimal threshold was used to generate the final predictions and confusion matrices.

The ROC curve demonstrates the trade-off between sensitivity (true positive rate) and specificity (1 - false positive rate) at various threshold settings. The area under the ROC curve (AUC) provides an aggregate measure of performance across all possible classification thresholds.

## Part 4: Linear SVM Regressor

In this part, we implement a regressor based on linear SVM and test it on the Bike Sharing dataset.

In [ ]:
def evaluate_svm_regressor(X, y, n_folds=6):
    # Initialize KFold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Initialize metrics storage
    mse_scores = []
    r2_scores = []
    times = []
    
    # For each fold
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train regressor
        start_time = time.time()
        reg = svm.SVR(kernel='linear')
        reg.fit(X_train, y_train)
        
        # Predict
        y_pred = reg.predict(X_test)
        end_time = time.time()
        
        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        execution_time = end_time - start_time
        
        # Store metrics
        mse_scores.append(mse)
        r2_scores.append(r2)
        times.append(execution_time)
        
        # Print metrics for the first fold (as required)
        if fold == 0:
            print(f"Fold 1 results:")
            print(f"Training/test split: {len(X_train)}/{len(X_test)} samples")
            print(f"Mean Squared Error (MSE): {mse:.4f}")
            print(f"R² Score: {r2:.4f}")
            print(f"Execution time: {execution_time:.4f} seconds")
            
            # Plot actual vs predicted for the first fold
            plt.figure(figsize=(10, 6))
            plt.scatter(y_test, y_pred, alpha=0.5)
            plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
            plt.xlabel('Actual')
            plt.ylabel('Predicted')
            plt.title('SVM Regressor: Actual vs Predicted (Fold 1)')
            plt.show()
    
    # Print average metrics
    print("\nAverage results across all folds:")
    print(f"Average MSE: {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")
    print(f"Average R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
    print(f"Average Execution time: {np.mean(times):.4f} ± {np.std(times):.4f} seconds")
    
    # Return all metrics
    return {
        'mse_scores': mse_scores,
        'r2_scores': r2_scores,
        'times': times
    }

In [ ]:
# Run Linear SVM Regressor
print("Linear SVM Regressor on Bike Sharing Dataset:")
svm_regressor_results = evaluate_svm_regressor(
    X_bike_scaled, 
    y_bike.values,
    n_folds=6
)

### Part 4 Results Summary

The linear SVM regressor has been evaluated on the Bike Sharing dataset using 6-fold cross-validation. The results show the regressor's performance in terms of Mean Squared Error (MSE), R² score, and execution time.

The scatter plot shows the relationship between actual and predicted values for the first fold, with the red dashed line representing perfect predictions. This helps visualize the model's performance.

## Part 5: Decision Tree Classifier

In this part, we implement a classifier based on Decision Trees and test it on the Breast Cancer dataset.
We'll experiment with two different pruning strategies.

In [ ]:
def extract_rules(tree, feature_names, class_names):
    """Extract rules from a decision tree"""
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != tree_.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != tree_.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            
            # Add left node (<=)
            path.append((name, "<=", threshold))
            recurse(tree_.children_left[node], path, paths)
            
            # Remove last item for the right branch
            path.pop()
            
            # Add right node (>)
            path.append((name, ">", threshold))
            recurse(tree_.children_right[node], path, paths)
            
            # Remove last item after both branches are processed
            path.pop()
        else:
            # Leaf node - get the class that has the majority
            class_idx = np.argmax(tree_.value[node])
            class_name = class_names[class_idx]
            paths.append((path.copy(), class_name))
            
    recurse(0, path, paths)
    
    # Convert to rules
    rules = []
    for path, class_name in paths:
        rule = "IF " + " AND ".join([f"{name} {inequality} {threshold:.4f}" for name, inequality, threshold in path])
        rule += f" THEN class = {class_name}"
        rules.append(rule)
    
    return rules

In [ ]:
def evaluate_dt_classifier(X, y, feature_names, class_names=[0, 1], n_folds=6):
    # Initialize KFold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Initialize metrics storage for different pruning strategies
    results = {
        'no_pruning': {
            'accuracies': [],
            'times': [],
            'model': None
        },
        'max_depth': {
            'accuracies': [],
            'times': [],
            'model': None
        },
        'min_samples_leaf': {
            'accuracies': [],
            'times': [],
            'model': None
        }
    }
    
    # For each fold
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train with no pruning
        start_time = time.time()
        dt_no_pruning = tree.DecisionTreeClassifier(random_state=42)
        dt_no_pruning.fit(X_train, y_train)
        y_pred_no_pruning = dt_no_pruning.predict(X_test)
        end_time = time.time()
        
        acc_no_pruning = accuracy_score(y_test, y_pred_no_pruning)
        time_no_pruning = end_time - start_time
        
        # Train with max_depth pruning
        start_time = time.time()
        dt_max_depth = tree.DecisionTreeClassifier(max_depth=5, random_state=42)
        dt_max_depth.fit(X_train, y_train)
        y_pred_max_depth = dt_max_depth.predict(X_test)
        end_time = time.time()
        
        acc_max_depth = accuracy_score(y_test, y_pred_max_depth)
        time_max_depth = end_time - start_time
        
        # Train with min_samples_leaf pruning
        start_time = time.time()
        dt_min_samples = tree.DecisionTreeClassifier(min_samples_leaf=5, random_state=42)
        dt_min_samples.fit(X_train, y_train)
        y_pred_min_samples = dt_min_samples.predict(X_test)
        end_time = time.time()
        
        acc_min_samples = accuracy_score(y_test, y_pred_min_samples)
        time_min_samples = end_time - start_time
        
        # Store metrics
        results['no_pruning']['accuracies'].append(acc_no_pruning)
        results['no_pruning']['times'].append(time_no_pruning)
        
        results['max_depth']['accuracies'].append(acc_max_depth)
        results['max_depth']['times'].append(time_max_depth)
        
        results['min_samples_leaf']['accuracies'].append(acc_min_samples)
        results['min_samples_leaf']['times'].append(time_min_samples)
        
        # Store models for first fold
        if fold == 0:
            results['no_pruning']['model'] = dt_no_pruning
            results['max_depth']['model'] = dt_max_depth
            results['min_samples_leaf']['model'] = dt_min_samples
            
            print(f"Fold 1 results:")
            print(f"Training/test split: {len(X_train)}/{len(X_test)} samples")
            print("\nNo Pruning:")
            print(f"Tree size (number of nodes): {dt_no_pruning.tree_.node_count}")
            print(f"Maximum depth: {dt_no_pruning.tree_.max_depth}")
            print(f"Accuracy: {acc_no_pruning:.4f}")
            print(f"Execution time: {time_no_pruning:.4f} seconds")
            
            print("\nMax Depth Pruning (max_depth=5):")
            print(f"Tree size (number of nodes): {dt_max_depth.tree_.node_count}")
            print(f"Maximum depth: {dt_max_depth.tree_.max_depth}")
            print(f"Accuracy: {acc_max_depth:.4f}")
            print(f"Execution time: {time_max_depth:.4f} seconds")
            
            print("\nMin Samples Leaf Pruning (min_samples_leaf=5):")
            print(f"Tree size (number of nodes): {dt_min_samples.tree_.node_count}")
            print(f"Maximum depth: {dt_min_samples.tree_.max_depth}")
            print(f"Accuracy: {acc_min_samples:.4f}")
            print(f"Execution time: {time_min_samples:.4f} seconds")
    
    # Print average metrics
    print("\nAverage results across all folds:")
    print(f"No Pruning - Average Accuracy: {np.mean(results['no_pruning']['accuracies']):.4f} ± {np.std(results['no_pruning']['accuracies']):.4f}")
    print(f"Max Depth Pruning - Average Accuracy: {np.mean(results['max_depth']['accuracies']):.4f} ± {np.std(results['max_depth']['accuracies']):.4f}")
    print(f"Min Samples Leaf Pruning - Average Accuracy: {np.mean(results['min_samples_leaf']['accuracies']):.4f} ± {np.std(results['min_samples_leaf']['accuracies']):.4f}")
    
    # Extract rules from the best pruning strategy model of the first fold
    best_strategy = max(['no_pruning', 'max_depth', 'min_samples_leaf'], 
                        key=lambda x: np.mean(results[x]['accuracies']))
    
    print(f"\nExtracting rules from the best model: {best_strategy}")
    best_model = results[best_strategy]['model']
    rules = extract_rules(best_model, feature_names, class_names)
    
    print(f"Number of rules: {len(rules)}")
    print("Sample rules (first 5):")
    for i, rule in enumerate(rules[:5]):
        print(f"Rule {i+1}: {rule}")
    
    # Visualize the best tree if it's small enough
    if best_model.tree_.node_count < 50:  # Only visualize if tree is not too large
        plt.figure(figsize=(20, 10))
        tree.plot_tree(best_model, feature_names=feature_names, class_names=[str(c) for c in class_names], filled=True)
        plt.title(f"Decision Tree Visualization ({best_strategy})")
        plt.show()
    
    return results, rules

In [ ]:
# Set feature names for the breast cancer dataset
cancer_feature_names = X_cancer.columns.tolist()

# Run Decision Tree Classifier
print("Decision Tree Classifier on Breast Cancer Dataset:")
dt_classifier_results, dt_rules = evaluate_dt_classifier(
    X_cancer_scaled, 
    y_cancer.values,
    feature_names=cancer_feature_names,
    class_names=['Benign (0)', 'Malignant (1)'],
    n_folds=6
)

### Part 5 Results Summary

The Decision Tree classifier has been evaluated on the Breast Cancer Wisconsin dataset using 6-fold cross-validation. We experimented with two different pruning strategies:

1. **Max Depth Pruning**: This strategy limits the maximum depth of the tree to 5 levels, preventing the tree from growing too deep and potentially overfitting. This reduces the complexity of the model and can improve generalization.

2. **Min Samples Leaf Pruning**: This strategy requires each leaf node to have at least 5 samples, which prevents the creation of very specific rules based on just a few training examples. This also helps prevent overfitting.

We also included a baseline model with no pruning to compare the effectiveness of the pruning strategies.

Additionally, we've extracted decision rules from the best-performing model. These rules provide a human-readable interpretation of the model's decision-making process, making it easier to understand how the model classifies samples.

## Part 6: Decision Tree Regressor

In this part, we implement a regressor based on Decision Trees and test it on the Bike Sharing dataset.

In [ ]:
def extract_regression_rules(tree, feature_names):
    """Extract rules from a regression tree"""
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != tree_.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != tree_.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            
            # Add left node (<=)
            path.append((name, "<=", threshold))
            recurse(tree_.children_left[node], path, paths)
            
            # Remove last item for the right branch
            path.pop()
            
            # Add right node (>)
            path.append((name, ">", threshold))
            recurse(tree_.children_right[node], path, paths)
            
            # Remove last item after both branches are processed
            path.pop()
        else:
            # Leaf node - get the prediction value
            value = tree_.value[node][0][0]  # Mean value of samples in this leaf
            paths.append((path.copy(), value))
            
    recurse(0, path, paths)
    
    # Convert to rules
    rules = []
    for path, value in paths:
        rule = "IF " + " AND ".join([f"{name} {inequality} {threshold:.4f}" for name, inequality, threshold in path])
        rule += f" THEN value = {value:.4f}"
        rules.append(rule)
    
    return rules

In [ ]:
def evaluate_dt_regressor(X, y, feature_names, n_folds=6):
    # Initialize KFold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Initialize metrics storage
    mse_scores = []
    r2_scores = []
    times = []
    best_model = None
    best_fold_r2 = -float('inf')
    
    # For each fold
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train regressor
        start_time = time.time()
        dt = tree.DecisionTreeRegressor(max_depth=10, random_state=42)
        dt.fit(X_train, y_train)
        
        # Predict
        y_pred = dt.predict(X_test)
        end_time = time.time()
        
        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        execution_time = end_time - start_time
        
        # Store metrics
        mse_scores.append(mse)
        r2_scores.append(r2)
        times.append(execution_time)
        
        # Keep track of the best model
        if r2 > best_fold_r2:
            best_fold_r2 = r2
            best_model = dt
        
        # Print metrics for the first fold (as required)
        if fold == 0:
            print(f"Fold 1 results:")
            print(f"Training/test split: {len(X_train)}/{len(X_test)} samples")
            print(f"Tree size (number of nodes): {dt.tree_.node_count}")
            print(f"Maximum depth: {dt.tree_.max_depth}")
            print(f"Mean Squared Error (MSE): {mse:.4f}")
            print(f"R² Score: {r2:.4f}")
            print(f"Execution time: {execution_time:.4f} seconds")
            
            # Plot actual vs predicted for the first fold
            plt.figure(figsize=(10, 6))
            plt.scatter(y_test, y_pred, alpha=0.5)
            plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
            plt.xlabel('Actual')
            plt.ylabel('Predicted')
            plt.title('Decision Tree Regressor: Actual vs Predicted (Fold 1)')
            plt.show()
    
    # Print average metrics
    print("\nAverage results across all folds:")
    print(f"Average MSE: {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")
    print(f"Average R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
    print(f"Average Execution time: {np.mean(times):.4f} ± {np.std(times):.4f} seconds")
    
    # Extract rules from the best model
    print(f"\nExtracting rules from the best model (fold with highest R²)")
    rules = extract_regression_rules(best_model, feature_names)
    
    print(f"Number of rules: {len(rules)}")
    print("Sample rules (first 5):")
    for i, rule in enumerate(rules[:5]):
        print(f"Rule {i+1}: {rule}")
    
    # Visualize tree if it's small enough
    if best_model.tree_.node_count < 50:  # Only visualize if tree is not too large
        plt.figure(figsize=(20, 10))
        tree.plot_tree(best_model, feature_names=feature_names, filled=True)
        plt.title("Decision Tree Regression Visualization")
        plt.show()
    
    return {
        'mse_scores': mse_scores,
        'r2_scores': r2_scores,
        'times': times,
        'rules': rules,
        'best_model': best_model
    }

In [ ]:
# Set feature names for the bike sharing dataset
bike_feature_names = X_bike.columns.tolist()

# Run Decision Tree Regressor
print("Decision Tree Regressor on Bike Sharing Dataset:")
dt_regressor_results = evaluate_dt_regressor(
    X_bike_scaled, 
    y_bike.values,
    feature_names=bike_feature_names,
    n_folds=6
)

### Part 6 Results Summary

The Decision Tree regressor has been evaluated on the Bike Sharing dataset using 6-fold cross-validation. We've configured the model with a maximum depth of 10 to prevent overfitting while still allowing it to capture the underlying patterns in the data.

The model's performance has been assessed using Mean Squared Error (MSE) and R² score metrics, which provide insights into both the absolute prediction errors and the proportion of variance explained by the model.

We've also extracted regression rules from the best-performing model (the one with the highest R² score). These rules provide a transparent representation of how the model makes predictions, showing the conditions that lead to different bike rental count predictions.

## Conclusions

In this assignment, we've implemented and evaluated various machine learning algorithms for classification and regression tasks:

1. **KNN Classifier with Euclidean distance**: We implemented a custom KNN classifier and evaluated it on the Breast Cancer dataset. The model demonstrated its ability to effectively classify tumors as benign or malignant.

2. **KNN Regressor with Manhattan distance**: Our custom KNN regressor was tested on the Bike Sharing dataset, showing how the k-nearest neighbors approach can be applied to regression problems.

3. **Linear SVM Classifier**: We utilized scikit-learn's SVM implementation for classification on the Breast Cancer dataset, examining the model's performance using ROC curves and finding optimal decision thresholds.

4. **Linear SVM Regressor**: We applied SVR to the Bike Sharing dataset, demonstrating how support vector methods can be extended to regression problems.

5. **Decision Tree Classifier**: We experimented with different pruning strategies (max depth and min samples leaf) to understand their impact on model complexity and performance for the Breast Cancer dataset.

6. **Decision Tree Regressor**: We applied decision trees to the regression task with the Bike Sharing dataset and extracted human-readable rules from the model.

For each model, we reported performance metrics, execution times, and visual representations of the results. We also implemented rule extraction for decision tree models, providing interpretable insights into the models' decision-making processes.

This comprehensive evaluation provides a solid foundation for understanding the strengths and weaknesses of each algorithm in different contexts, as well as the impact of various parameter choices and pruning strategies on model performance.